In [12]:
import re
import logging
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [20]:
def extract_signals(text: str):
    """Enhanced signal extraction to handle more cases."""
    keywords = {
        'begin', 'end', 'if', 'else', 'case', 'endcase',
        'always', 'always_ff', 'always_comb', 'always_latch',
        'posedge', 'negedge', 'wire', 'reg', 'logic', 'assign',
        'parameter', 'property', 'endproperty', 'sequence', 'endsequence',
        'or', 'and', 'not', 'xor', 'nor', 'nand',
        'disable', 'iff',
        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'
    }
    
    # First remove sensitivity list and disable condition
    text = re.sub(r'@\s*\([^)]*\)', ' ', text)  # Remove @(posedge clk)
    text = re.sub(r'disable\s+iff\s*\([^)]*\)', ' ', text)  # Remove disable iff(...)
    
    # Remove SystemVerilog delay operators
    text = re.sub(r'##\d+', ' ', text)
    
    # Remove common operators
    text = re.sub(r'[!&|=<>~\^\+\-\*/%\?:]', ' ', text)
    
    # Remove parentheses but keep their contents
    text = text.replace('(', ' ').replace(')', ' ')
    
    # Match valid SystemVerilog identifiers
    signal_pattern = r'\b([a-zA-Z_]\w*(?:\s*\[[^\]]+\])?)\b'
    
    signals = set()
    for match in re.finditer(signal_pattern, text):
        signal = match.group(1)
        signal = signal.strip()
        base_signal = signal.split('[')[0].strip()
        
        if (base_signal not in keywords and 
            not base_signal.isdigit() and
            not any(c in base_signal for c in '+-*/%')):
            signals.add(signal)
    
    logger.debug(f"Extracted signals from text: {text}")
    logger.debug(f"Found signals: {signals}")
    
    return signals

In [21]:
def debug_signal_extraction(text: str):
    """Debug helper for signal extraction process."""
    logger.debug("Signal Extraction Debug:")
    logger.debug(f"Original text: {text}")
    
    # Show text after each preprocessing step
    text_no_delays = re.sub(r'##\d+', ' ', text)
    logger.debug(f"After removing delays: {text_no_delays}")
    
    text_no_edges = re.sub(r'@\s*\([^)]+\)', ' ', text_no_delays)
    logger.debug(f"After removing edge operators: {text_no_edges}")
    
    text_no_operators = re.sub(r'[!&|=<>~\^\+\-\*/%\?:]', ' ', text_no_edges)
    logger.debug(f"After removing operators: {text_no_operators}")
    
    text_no_parens = re.sub(r'\([^)]*\)', ' ', text_no_operators)
    logger.debug(f"After removing parentheses: {text_no_parens}")
    
    # Show found signals
    signals = extract_signals(text)
    logger.debug(f"Final extracted signals: {signals}")

In [23]:
logger.setLevel(logging.DEBUG)

text = "    property address_stability; @(posedge pclk) disable iff(!preset_n) (psel && !penable) |=> $stable(paddr);endproperty"
debug_signal_extraction(text)

2024-12-07 22:44:48,214 - DEBUG - Signal Extraction Debug:
2024-12-07 22:44:48,216 - DEBUG - Original text:     property address_stability; @(posedge pclk) disable iff(!preset_n) (psel && !penable) |=> $stable(paddr);endproperty
2024-12-07 22:44:48,216 - DEBUG - After removing delays:     property address_stability; @(posedge pclk) disable iff(!preset_n) (psel && !penable) |=> $stable(paddr);endproperty
2024-12-07 22:44:48,217 - DEBUG - After removing edge operators:     property address_stability;   disable iff(!preset_n) (psel && !penable) |=> $stable(paddr);endproperty
2024-12-07 22:44:48,219 - DEBUG - After removing operators:     property address_stability;   disable iff( preset_n) (psel     penable)     $stable(paddr);endproperty
2024-12-07 22:44:48,221 - DEBUG - After removing parentheses:     property address_stability;   disable iff        $stable ;endproperty
2024-12-07 22:44:48,222 - DEBUG - Extracted signals from text:     property address_stability;      psel     penable  